<a href="https://colab.research.google.com/github/miuceo/Tashkent_housing_prics_prediction/blob/main/Machine_Learning_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import sklearn

df = pd.read_csv("https://raw.githubusercontent.com/miuceo/Tashkent_housing_prics_prediction/refs/heads/main/Dataset.csv",  usecols = ["district", 'rooms', 'size', 'level', 'max_levels', 'psp', 'price',])
df.head()

,district,rooms,size,level,max_levels,price,psp
0,Yunusobod,3,57.0,4,4,52000.0,912.280702
1,Yakkasaroy,2,52.0,4,5,56000.0,1076.923077
2,Chilonzor,2,42.0,4,4,37000.0,880.952381
3,Chilonzor,3,65.0,1,4,49500.0,761.538462
4,Chilonzor,3,70.0,3,5,55000.0,785.714286


In [14]:
from sklearn.model_selection import StratifiedShuffleSplit
stratified_split = StratifiedShuffleSplit(n_splits=1, test_size = 0.2, random_state=42)
for train_index, test_index in stratified_split.split(df, df['district']):
    train_set = df.loc[train_index]
    test_set = df.loc[test_index]

In [18]:
X = train_set.drop('price', axis = 1)
X_num = X.drop('district', axis = 1)
Y = train_set['price'].copy()

In [23]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

num_attribs = list(X_num)
cat_attribs = ['district']

num_pipline = Pipeline([
    ('std_scaler', StandardScaler())
])

full_pipline = ColumnTransformer([
    ("cat", OneHotEncoder(), cat_attribs),
    ("num_pip", num_pipline, num_attribs)
])

In [24]:
X_prepared = full_pipline.fit_transform(X)
X_prepared[0]

array([ 0.        ,  0.        ,  0.        ,  1.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        , -0.57294668, -0.03519322, -0.76303357,
       -0.77052632, -0.10086912])

In [25]:
from sklearn.linear_model import LinearRegression
LR_model = LinearRegression()

LR_model.fit(X_prepared, Y)

LinearRegression()

In [31]:
test_data = test_set

In [32]:
X_test = test_data.drop("price", axis = 1)
Y_test = test_data['price'].copy()

X_test_prepared = full_pipline.fit_transform(X_test)

In [35]:
test_predicted = LR_model.predict(X_test_prepared)
pd.DataFrame({"Bashorat" : test_predicted, "Asl qiymat" : Y_test}).sample(20)

,Bashorat,Asl qiymat
5941,71395.459317,42000.0
834,56298.375269,56500.0
2696,42236.240086,33000.0
2585,38149.705537,28000.0
3961,41214.544455,34500.0
1598,67026.531335,39000.0
3579,44675.623738,40000.0
1077,49857.504811,47000.0
1286,96872.997863,52000.0
3170,72927.422057,63000.0


In [39]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

mse = mean_squared_error(Y_test, test_predicted)
mae = mean_absolute_error(Y_test, test_predicted)
print(f"MAE : {mae}\nMSE : {np.sqrt(mse)}")

50513.59510707094

In [50]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(LR_model, X_prepared, Y, scoring = "neg_mean_squared_error", cv = 8)

def dispaly_scores(scores):
    print(f"Scores : {scores}")
    print(f"Mean score : {scores.mean()}")
    print(f"STD : {scores.std()}")

dispaly_scores(np.sqrt(-scores))

Scores : [ 26545.43262706 158679.05272181 257647.36361584  89371.35275553
  24284.08090424 119624.18286495  33356.2507713  114323.47405663]
Mean score : 102978.89878966843
STD : 74580.5215584955


In [52]:
from sklearn.ensemble import RandomForestRegressor

RF_model = RandomForestRegressor()
RF_model.fit(X_prepared, Y)

RandomForestRegressor()

In [54]:
test_rf_predicted = RF_model.predict(X_test_prepared)
pd.DataFrame({"Bashorat" : test_rf_predicted, "Asl qiymat" : Y_test})

,Bashorat,Asl qiymat
4844,31201.000000,30500.0
13,24692.000000,21200.0
954,34809.000000,36000.0
4539,51773.712385,79000.0
4017,43031.150000,42200.0
...,...,...
6342,84725.114858,93500.0
2620,46380.175804,38000.0
5311,53985.000000,53000.0
5780,36230.000000,34000.0


In [55]:
mse = mean_squared_error(Y_test, test_predicted)
mae = mean_absolute_error(Y_test, test_predicted)
print(f"MAE : {mae}\nMSE : {np.sqrt(mse)}")

MAE : 50513.59510707094
MSE : 1122872.2642000732


In [59]:
scores_rf = cross_val_score(RF_model, X_prepared, Y, scoring = "neg_mean_squared_error", cv = 10)
dispaly_scores(np.sqrt(-scores_rf))

Scores : [  5892.7034085    5715.74856006 163773.24933268 428491.57598432
  31653.0813783    1489.79829949  29567.71387073  72320.83856815
  42481.90862954   5763.02727162]
Mean score : 78714.96453033877
STD : 125491.13941232896


In [60]:
import pickle

file_name = "RR_model.pkl"
with open(file_name, "wb") as file:
    pickle.dump(RF_model, file)